In [5]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key = os.getenv("GROQ_API_KEY")
)

In [6]:
response = llm.invoke("What is the capital of France?")
print(response.content)

The capital of France is Paris.


In [7]:
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="my_collection")

In [8]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

In [9]:
all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This is a document about pineapple',
  'This is a document about oranges'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [10]:
collection.add(
    documents=["Facebook is a social media platform"],
    ids=["id3"]
)

In [11]:
results = collection.query(
    query_texts=['Query is about social media'],
    n_results=2
)
results

{'ids': [['id3', 'id1']],
 'embeddings': None,
 'documents': [['Facebook is a social media platform',
   'This is a document about pineapple']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[0.7233924269676208, 1.664230465888977]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [12]:
collection.delete(ids=all_docs['ids'])

In [13]:
collection.add(
    documents=["Facebook is a social media platform"],
    ids=["id1"],
    metadatas={"url": "www.facebook.com"}
)

In [14]:
collection.get(ids=["www.facebook.com"])

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [15]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-42816?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware EngineerBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more than out

In [34]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `Company's name`, `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res.content

'[\n  {\n    "Company\'s name": "NIKE, INC.",\n    "role": "Software Engineer",\n    "experience": "2 years of experience in the job offered or in a Engineering-related occupation",\n    "skills": [\n      "Java",\n      "CI/CD Pipelines (Jenkins)",\n      "Springboot",\n      "DevOps",\n      "AWS infrastructure",\n      "Microservice Architecture",\n      "SQL / NoSQL Databases",\n      "Distributed Systems",\n      "Observability Tooling (Splunk / SignalFx)"\n    ],\n    "description": "Develop, code/configure, test programs systems and solutions problems in order to meet defined digital product specifications and direction. Develop robust software solutions that have a direct impact on the business. Design and build scalable software solutions, implement and integrate new technologies, manage and evolve existing platforms. Design and implement scalable applications that leverage prediction models and optimization programs to deliver data driven decisions that result in immense busi

In [35]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{"Company's name": 'NIKE, INC.',
  'role': 'Software Engineer',
  'experience': '2 years of experience in the job offered or in a Engineering-related occupation',
  'skills': ['Java',
   'CI/CD Pipelines (Jenkins)',
   'Springboot',
   'DevOps',
   'AWS infrastructure',
   'Microservice Architecture',
   'SQL / NoSQL Databases',
   'Distributed Systems',
   'Observability Tooling (Splunk / SignalFx)'],
  'description': 'Develop, code/configure, test programs systems and solutions problems in order to meet defined digital product specifications and direction. Develop robust software solutions that have a direct impact on the business. Design and build scalable software solutions, implement and integrate new technologies, manage and evolve existing platforms. Design and implement scalable applications that leverage prediction models and optimization programs to deliver data driven decisions that result in immense business impact. Assess a well-defined problem and lead the development of

In [36]:
type(json_res)

list

In [37]:
import pandas as pd
portfolios = pd.read_csv('my_portfolio.csv')
portfolios

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [38]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in portfolios.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [39]:
job = json_res[0]
job["description"]

'Develop, code/configure, test programs systems and solutions problems in order to meet defined digital product specifications and direction. Develop robust software solutions that have a direct impact on the business. Design and build scalable software solutions, implement and integrate new technologies, manage and evolve existing platforms. Design and implement scalable applications that leverage prediction models and optimization programs to deliver data driven decisions that result in immense business impact. Assess a well-defined problem and lead the development of a technical solution that meets the needs of the business and aligns with architectural standards. Solve business problems at scale using software solutions and develop products focused primarily for the supply chain.'

In [40]:
links = collection.query(query_texts=job['skills'], n_results=1).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'}]]

In [41]:
job['skills']

['Java',
 'CI/CD Pipelines (Jenkins)',
 'Springboot',
 'DevOps',
 'AWS infrastructure',
 'Microservice Architecture',
 'SQL / NoSQL Databases',
 'Distributed Systems',
 'Observability Tooling (Splunk / SignalFx)']

In [42]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Ghassen, a business development executive at CherSolutions. CherSolutions is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client {Company_name} with name  regarding the job mentioned above describing the capability of 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase CherSolutions's portfolio: {link_list}
        Remember you are Ghassen, BDE at CherSolutions. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": job["description"], "link_list": links , "Company_name": job["Company's name"]})
print(res.content)

Subject: Unlocking Supply Chain Efficiency with AI-Powered Solutions at NIKE, INC.

Dear Hiring Manager at NIKE, INC.,

I came across the job description for a developer position at NIKE, INC., and I am excited to introduce CherSolutions, an AI & Software Consulting company that can help you achieve your digital product goals. Our team of experts has a proven track record of developing robust software solutions that drive business impact.

At CherSolutions, we specialize in designing and building scalable software solutions that leverage prediction models and optimization programs to deliver data-driven decisions. Our expertise in developing applications that drive immense business impact aligns perfectly with your requirements. We have a deep understanding of assessing well-defined problems and leading the development of technical solutions that meet business needs and align with architectural standards.

Our portfolio showcases our capabilities in developing software solutions that d